In [0]:
# load packages
import pandas as pd
import numpy as np
import glob
import errno
import re
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
#from pattern.nl import sentiment
from nltk.stem.snowball import SnowballStemmer
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import *
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# create empty dataframe
#df = pd.DataFrame(columns=['text', 'sentiment'])

In [0]:
PATH = '/content/drive/My Drive/train'

In [0]:
# add text and sentiment to dataframe
'''
path = f'{PATH}/pos_t/*.txt'
files = glob.glob(path)
i = 0
for name in files:
    try:
        if(i %100==0):
          print(i)
        with open(name,encoding="utf8") as f:
            text = f.read().replace('\n', '').lower()
            df = df.append({'text': text, 'sentiment': 1}, ignore_index=True)
            i = i+1
    except IOError as exc:
        if exc.errno != errno.EISDIR:
            raise
'''

In [0]:
x = df['text']
print(len(x))
print(x)

In [0]:
'''
path = f'{PATH}/neg_t/*.txt'
files = glob.glob(path)
i = 0
for name in files:
    try:
        if(i %100==0):
          print(i)
        with open(name,encoding="utf8") as f:
            text = f.read().replace('\n', '').lower()
            df = df.append({'text': text, 'sentiment': 0}, ignore_index=True)   
            i = i+1
    except IOError as exc:
        if exc.errno != errno.EISDIR:
            raise
'''

In [0]:
# shuffle data
#df = df.sample(frac=1)

In [0]:
df = pd.read_csv(f'{PATH}/data.csv')
df_test = pd.read_csv(f'{PATH}/dataTest.csv')

In [0]:
# get x and y
x = df['text']
X_test = df_test['text']
y = df['sentiment']
Y_test = df_test['sentiment']
# split
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.01, random_state=42)

In [0]:
# create a dutch stemmer
stemmer = SnowballStemmer("dutch")
# Stemming
X_train = [stemmer.stem(x) for x in X_train]

X_test = [stemmer.stem(x) for x in X_test]

In [0]:
# create tokenizer instance
tokenizer = Tokenizer()   
# create mapping
tokenizer.fit_on_texts(X_train)
# map text to list of integer values
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [0]:

vocab_size = len(tokenizer.word_index) + 1
# all text to same length
maxlen = 320 
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [0]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open(f'{PATH}/320/wikipedia-320.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [0]:
embedding_matrix = zeros((vocab_size, 320))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

## Convolution

In [0]:
model = Sequential()

embedding_layer = Embedding(vocab_size, 320, weights=[embedding_matrix], input_length=maxlen , trainable=False)  
model.add(embedding_layer)

model.add(Conv1D(256, 5, activation='relu'))  #Play with window size (1,3,5,7)   # Test with more neurons first test 256 n second test 2nd layer (64/128) 3th test with batchnorm or other norm 4th play with windowsize
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))   # test with relu
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [0]:
earlyStop= EarlyStopping(monitor='val_loss', patience=6, verbose=0, restore_best_weights=True)

history = model.fit(X_train, y_train, batch_size=256, epochs=20, verbose=2, validation_split=0.15, callbacks=[earlyStop])  # play around with bathsize


In [0]:
score = model.evaluate(X_test, Y_test, verbose=1)

2226/2226 [==============================] - 0s 95us/step


In [0]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

Test Score: 0.11368543745051175
Test Accuracy: 0.9725965855899205


In [0]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc = 'upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc = 'upper left')
plt.show()

In [0]:
## TIME TO CLASSIFY
df_current = pd.read_csv(f'{PATH}/gtst_subtitles_5491_emo.csv')
df_true = pd.read_csv(f'{PATH}/gtst_subtitles_5491_true.csv', sep=';')


In [0]:
df_current.head()

In [0]:
# get the labels
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as score

true_labels = df_true['true']
true_labels = true_labels[1:]
true_labels = [x for x in true_labels]
current_labels = df_current['sentiment_polarity']
current_labels = current_labels[1:]
current_labels = [x for x in current_labels]
text = df_true['text']
text = text[1:]
text = [x.replace('\x03', '') for x in text]
text = [x.replace('\x06', '') for x in text]
text = [x.replace('\x07', '') for x in text]
text = [x.replace('.', ' ') for x in text]
text = [x.replace(',', ' ') for x in text]

text2 = [stemmer.stem(x) for x in text]

In [0]:
textInput = tokenizer.texts_to_sequences(text)
textInput = pad_sequences(textInput, padding='post', maxlen=maxlen)
textInput2 = tokenizer.texts_to_sequences(text2)
textInput2 = pad_sequences(textInput2, padding='post', maxlen=maxlen)
model_labels = model.predict(textInput2)
model_labels = [x[0] for x in model_labels]


In [0]:
def currentToClass(x,d):
  if(x<(-1+d)):
    return 0
  elif(x>(1-d)):
    return 2
  else:
    return 1

def modelToClass(x,d):
  if(x<d):
    return 0
  elif(x>(1-d)):
    return 2
  else: 
    return 1

In [0]:
dM = 0
dC = 0

d = 0.025
bscore = 0
while(d<0.5):
  labels = [modelToClass(x,d) for x in model_labels]
  precision, recall, fscore, support = score(true_labels, labels)
  avgFM = np.mean(fscore)
  if(avgFM>=bscore):
    bscore = avgFM
    dM = d
  d = d+0.025

d = 0.05
bscore = 0
while(d<1):
  labels = [currentToClass(x,d) for x in current_labels]
  precision, recall, fscore, support = score(true_labels, labels)
  avgFC = np.mean(fscore)
  if(avgFC>=bscore):
    bscore = avgFC
    dC = d
  d = d+0.05

print(dC)
print(dM)
#current_labels = [currentToClass(x) for x in current_labels]

In [0]:
current_labels = [currentToClass(x,dC) for x in current_labels]
print('current labels')
precision, recall, fscore, support = score(true_labels, current_labels)
gemFC = np.mean(fscore)
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))

model_labels = [modelToClass(x,dM) for x in model_labels]
print(' ')
print('new model score')
precision, recall, fscore, support = score(true_labels, model_labels)
gemFM = np.mean(fscore)
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))

In [0]:
print(gemFC)
print(gemFM)